In [1]:
from mpi4py import MPI
from petsc4py import PETSc

import numpy as np

import basix
import ufl
import ufl.algorithms
from dolfinx import fem, mesh
from dolfinx_external_operator import FEMExternalOperator, replace_external_operators, evaluate_operands, evaluate_external_operators
from ufl import Measure, TestFunction, TrialFunction, derivative, grad, inner


In [2]:
domain = mesh.create_unit_square(MPI.COMM_WORLD, 2, 2)

gdim = domain.geometry.dim
V = fem.functionspace(domain, ("CG", 1))
v = ufl.TestFunction(V)

u_hat = ufl.TrialFunction(V)
u = fem.Function(V)
u.x.array[:] = 2.0  # in order to get non-zero forms after assembling

In [3]:
def N_external(u):
    return np.reshape(u**2, -1)

def dNdu_external(u):
    return np.reshape(2 * u, -1)

In [4]:
def Ns_external(derivatives):
    if derivatives == (0,):
        return N_external
    elif derivatives == (1,):
        return dNdu_external
    else:
        return NotImplementedError

In [5]:
V = fem.functionspace(domain, ("CG", 1))
v = ufl.TestFunction(V)
u_hat = ufl.TrialFunction(V)
u = fem.Function(V)

quadrature_degree=1
Qe = basix.ufl.quadrature_element(domain.topology.cell_name(), degree=quadrature_degree, value_shape=())
Q = fem.functionspace(domain, Qe)
dx_m = ufl.Measure("dx", domain=domain, metadata={"quadrature_degree": quadrature_degree, "quadrature_scheme": "default"})

In [6]:
N = FEMExternalOperator(u, function_space=Q, external_function=None)
F = ufl.inner(N, v) * dx_m
J = ufl.derivative(F, u, u_hat)
F_replaced, F_ex_ops_list = replace_external_operators(F)
F_dolfinx = fem.form(F_replaced)

In [7]:
J_expanded = ufl.algorithms.expand_derivatives(J)
J_replaced, J_ex_ops_list = replace_external_operators(J_expanded)
J_dolfinx = fem.form(J_replaced)

()
()
()
()
()
v_1 * f
v_1 * f
()


In [8]:
quadrature_degree=1
Qe = basix.ufl.quadrature_element(domain.topology.cell_name(), degree=quadrature_degree, value_shape=(2,))
Q = fem.functionspace(domain, Qe)
dx_m = ufl.Measure("dx", domain=domain, metadata={"quadrature_degree": quadrature_degree, "quadrature_scheme": "default"})

In [9]:
N = FEMExternalOperator(u, function_space=Q, external_function=None)
F = ufl.inner(N, ufl.grad(v)) * dx_m
J = ufl.derivative(F, u, u_hat)
F_replaced, F_ex_ops_list = replace_external_operators(F)
F_dolfinx = fem.form(F_replaced)

In [10]:
J_expanded = ufl.algorithms.expand_derivatives(J)
J_replaced, J_ex_ops_list = replace_external_operators(J_expanded)
J_dolfinx = fem.form(J_replaced)

()
(2,)
(Index(9),)
()
(Index(9),)
{ A | A_{i_{10}} = f[i_{10}] * v_1 }
{ A | A_{i_9} = f[i_9] * v_1 }
(2,)


In [11]:
N.ref_coefficient.ufl_shape

(2,)

In [13]:
J_ex_ops_list[0].ufl_shape

(2,)

In [37]:
V = fem.functionspace(domain, ("CG", 1))
v = ufl.TestFunction(V)
u_hat = ufl.TrialFunction(V)
u = fem.Function(V)

quadrature_degree=1
Qe = basix.ufl.quadrature_element(domain.topology.cell_name(), degree=quadrature_degree, value_shape=(2,))
Q = fem.functionspace(domain, Qe)
dx_m = ufl.Measure("dx", domain=domain, metadata={"quadrature_degree": quadrature_degree, "quadrature_scheme": "default"})

In [38]:
# quadrature_points = basix.make_quadrature(basix.CellType.triangle, quadrature_degree, basix.QuadratureType.Default)
# num_cells = domain.topology.index_map(domain.topology.dim).size_local
# num_gauss_points = quadrature_points[0].shape[0]

In [39]:
sigma = ufl.grad(u)
N = FEMExternalOperator(sigma, function_space=Q, external_function=None)

In [40]:
F = ufl.inner(N, ufl.grad(v)) * dx_m
J = ufl.derivative(F, u, u_hat)

In [41]:
F_replaced, F_ex_ops_list = replace_external_operators(F)
F_dolfinx = fem.form(F_replaced)

In [42]:
N.arguments()

(Coargument(DualSpace(Mesh(blocked element (Basix element (P, triangle, 1, gll_warped, unset, False), (2,)), 1), blocked element (QuadratureElement(triangle, array([[0.33333333, 0.33333333]]), array([0.5]), IdentityPullback()), (2,))), 0, None),)

In [43]:
J_expanded = ufl.algorithms.expand_derivatives(J)
J_replaced, J_ex_ops_list = replace_external_operators(J_expanded)
J_dolfinx = fem.form(J_replaced)

(2,)
(2, 2)
(Index(23), Index(24))
(Index(24),)
(Index(23),)
{ A | A_{i_{23}} = sum_{i_{24}} f[i_{23}, i_{24}] * (grad(v_1))[i_{24}]  }
(2,)


In [15]:
N.ufl_operands[0]

Coefficient(FunctionSpace(Mesh(blocked element (Basix element (P, triangle, 1, gll_warped, unset, False), (2,)), 0), Basix element (P, triangle, 1, gll_warped, unset, False)), 1)

In [16]:
V.mesh

In [18]:
N.ref_function_space.mesh

In [14]:
def f(i,j):
    a = np.ones((2,2))
    print(a[i,j])

In [15]:
f(*tup)

1.0


In [25]:
var = np.ones((10))
var2 = var

In [31]:
var = np.full((10),2)

In [32]:
var2

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [24]:
var.__

'[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]'

In [20]:
type(ufl.grad(u))

ufl.differentiation.Grad

In [21]:
type(u)

dolfinx.fem.function.Function

In [19]:
u.function_space

FunctionSpace(Mesh(blocked element (Basix element (P, triangle, 1, gll_warped, unset, False), (2,)), 0), Basix element (P, triangle, 1, gll_warped, unset, False))

In [16]:
print(J_expanded)

Action({ conj((sum_{i_8} (grad(v_0))[i_8] * (conj((v_1[i_8]))) )) } * dx(<Mesh #0>[everywhere], {'quadrature_degree': 1, 'quadrature_scheme': 'default'}), ∂e(grad(f); grad(v_1), v_0)/∂o1)


In [17]:
print(J_replaced)

{ conj((sum_{i_8} (grad(v_0))[i_8] * (conj((({ A | A_{i_9} = sum_{i_{10}} f[i_9, i_{10}] * (grad(v_1))[i_{10}]  })[i_8]))) )) } * dx(<Mesh #0>[everywhere], {'quadrature_degree': 1, 'quadrature_scheme': 'default'})


In [18]:
print(N)

e(grad(f); v_0)


In [19]:
dN = J_ex_ops_list[0]

In [20]:
print(dN)

∂e(grad(f); grad(v_1), v_0)/∂o1


In [21]:
dN.ufl_shape

(2,)

In [22]:
N.ref_coefficient.x.array.reshape((num_cells, num_gauss_points, 2)).shape


(8, 1, 2)

In [23]:
dN.ufl_shape

(2,)

In [24]:
dN.ref_coefficient.x.array.reshape((num_cells, num_gauss_points, 2, 2)).shape


(8, 1, 2, 2)

In [25]:
evaluated_operands = evaluate_operands(F_ex_ops_list)
evaluated_operands

{Grad(Coefficient(FunctionSpace(Mesh(blocked element (Basix element (P, triangle, 1, gll_warped, unset, False), (2,)), 0), Basix element (P, triangle, 1, gll_warped, unset, False)), 1)): array([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])}

In [26]:
Qe = basix.ufl.quadrature_element(domain.topology.cell_name(), degree=quadrature_degree, value_shape=(2,2))
Q = fem.functionspace(domain, Qe)
dx_m = ufl.Measure("dx", domain=domain, metadata={"quadrature_degree": quadrature_degree, "quadrature_scheme": "default"})

sigma = ufl.grad(u)
N = FEMExternalOperator(sigma, function_space=Q, external_function=None)

F = ufl.inner(N, ufl.grad(ufl.grad(v))) * dx_m
J = ufl.derivative(F, u, u_hat)

In [27]:
F_replaced, F_ex_ops_list = replace_external_operators(F)
F_dolfinx = fem.form(F_replaced)
J_expanded = ufl.algorithms.expand_derivatives(J)
# J_dolfinx = fem.form(J_replaced)

In [28]:
print(J_expanded)

Action({ conj((sum_{i_{12}} sum_{i_{11}} (grad(grad(v_0)))[i_{11}, i_{12}] * (conj((v_1[i_{11}, i_{12}])))  )) } * dx(<Mesh #0>[everywhere], {'quadrature_degree': 1, 'quadrature_scheme': 'default'}), ∂e(grad(f); grad(v_1), v_0)/∂o1)


In [29]:
J_replaced, J_ex_ops_list = replace_external_operators(J_expanded)


(2,)
(2, 2, 2)
(Index(13), Index(14), Index(15))
(Index(15),)
(Index(13), Index(14))
{ A | A_{i_{13}, i_{14}} = sum_{i_{15}} f[i_{13}, i_{14}, i_{15}] * (grad(v_1))[i_{15}]  }
(2, 2)


In [31]:
Qe = basix.ufl.quadrature_element(domain.topology.cell_name(), degree=quadrature_degree, value_shape=(2,2))
Q = fem.functionspace(domain, Qe)
dx_m = ufl.Measure("dx", domain=domain, metadata={"quadrature_degree": quadrature_degree, "quadrature_scheme": "default"})

N = FEMExternalOperator(ufl.grad(ufl.grad(u)), function_space=Q, external_function=None)

F = ufl.inner(N, ufl.grad(ufl.grad(v))) * dx_m
J = ufl.derivative(F, u, u_hat)

In [32]:
F_replaced, F_ex_ops_list = replace_external_operators(F)
F_dolfinx = fem.form(F_replaced)
J_expanded = ufl.algorithms.expand_derivatives(J)
# J_dolfinx = fem.form(J_replaced)

In [33]:
J_replaced, J_ex_ops_list = replace_external_operators(J_expanded)


(2, 2)
(2, 2, 2, 2)
(Index(18), Index(19), Index(20), Index(21))
(Index(20), Index(21))
(Index(18), Index(19))
{ A | A_{i_{18}, i_{19}} = sum_{i_{21}} sum_{i_{20}} f[i_{18}, i_{19}, i_{20}, i_{21}] * (grad(grad(v_1)))[i_{20}, i_{21}]   }
(2, 2)
